In [47]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [48]:
mnist = input_data.read_data_sets("MNIST-dataset/",one_hot=True)

Extracting MNIST-dataset/train-images-idx3-ubyte.gz
Extracting MNIST-dataset/train-labels-idx1-ubyte.gz
Extracting MNIST-dataset/t10k-images-idx3-ubyte.gz
Extracting MNIST-dataset/t10k-labels-idx1-ubyte.gz


In [49]:
# HELPER
# INIT WEIGHTS

In [50]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

In [51]:
# INIT BIAS
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

In [52]:
# CONV2D
def conv2d(x,W):
    # x --> [batch, H, W, channels]
    # W --> [filter H, filter W, Channels IN, Channels OUT]
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

In [53]:
# POOLING
def max_pool_2by2(x):
    # x --> [batch,h,w,c]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1], padding='SAME')

In [54]:
# CONVOLUTIONAL LAYER
def convolutional_layer(input_x,shape):
    W = init_wheights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+b)

In [55]:
# NORMAL (FULLY CONNECTED)
def normal_full_layer(input_layer,size):
    input_size = int(input_layer.get_shape()[1])
    W = init_wheights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer,W)+b

In [56]:
# PLACEHOLDERS
x = tf.placeholder(tf.float32,shape=[None,784])

In [57]:
y_true = tf.placeholder(tf.float32,shape=[None,10])

In [58]:
# LAYERS (transformando os dados em imagem)
x_image = tf.reshape(x,[-1,28,28,1])
convo_1 = convolutional_layer(x_image,shape=[5,5,1,32])
convo_1_pooling = max_pool_2by2(convo_1)
convo_2 = convolutional_layer(convo_1_pooling, shape=[5,5,32,64])
convo_2_pooling = max_pool_2by2(convo_2)
convo_2_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [59]:
# DROPOUT
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob=hold_prob)
y_pred = normal_full_layer(full_one_dropout,10)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [61]:
# LOSS FUNCTION
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

In [63]:
# OPTIMIZER
optmizer = tf.train.AdadeltaOptimizer(learning_rate=0.001)
train = optmizer.minimize(cross_entropy)

In [64]:
init = tf.global_variables_initializer()

In [ ]:
steps = 5000
with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):
        batch_x, batch_y = mnist.train.next_batch(50)
        sess.run(train, feed_dict={x:batch_x, y_true:batch_y, hold_prob:0.5})
        if i%100 == 0:
            print("ON STEP: {}".format(i))
            print("ACCURACY: ")
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
            print("\n")

ON STEP: 0
ACCURACY: 
0.1243


ON STEP: 100
ACCURACY: 
0.1307


ON STEP: 200
ACCURACY: 
0.1336


ON STEP: 300
ACCURACY: 
0.1369


ON STEP: 400
ACCURACY: 
0.1387


ON STEP: 500
ACCURACY: 
0.1404


ON STEP: 600
ACCURACY: 
0.1401


ON STEP: 700
ACCURACY: 
